In [16]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
from PIL import Image
import numpy as np

In [6]:
key = get_random_bytes(16)
image = Image.open('tux.ppm')

In [14]:
def encrypt(method, data):
    iv = get_random_bytes(16)
    data = pad(data, AES.block_size)

    match method:
        case 'ECB':
            cipher = AES.new(key, AES.MODE_ECB)
            encrypted = cipher.encrypt(data)
        case 'CBC':
            cipher = AES.new(key, AES.MODE_CBC, iv)
            encrypted = cipher.encrypt(data)
    return encrypted


In [32]:
image_array = np.array(image)
# We want to preserve the original format to visualize the encrypted images.
width, height = image.size
mode = image.mode 

ecb_encrypted = encrypt('ECB', image.tobytes())
cbc_encrypted = encrypt('CBC', image.tobytes())

ecb_image = Image.new(mode, (width, height))
ecb_image.frombytes(ecb_encrypted)

cbc_image = Image.new(mode, (width, height))
cbc_image.frombytes(cbc_encrypted)

ecb_image.save('tux_ecb.png')
cbc_image.save('tux_cbc.png')


> **Note:** Saved as png to render them properly in markdown, would still work with ppm.

<table>
  <tr>
    <td><img src="./tux_ecb.png" alt="ECB" width="300"/></td>
    <td><img src="./tux_cbc.png" alt="CBC" width="300"/></td>
  </tr>
  <tr>
    <td style="text-align: center;">ECB Mode</td>
    <td style="text-align: center;">CBC Mode</td>
  </tr>
</table>

### ¿Por qué el cifrado ECB revela los patrones de la imagen?

El cifrado ECB (Electronic Codebook) revela los patrones de la imagen porque cifra cada bloque de la imagen de forma independiente con la misma clave. Si la imagen contiene áreas grandes con el mismo color o patrón, estos bloques idénticos se cifrarán exactamente de la misma manera, resultando en bloques cifrados idénticos. 

### ¿Cómo cambia la apariencia con CBC?

En CBC, cada bloque se cifra dependiendo del bloque anterior, utilizando un vector de inicialización (IV) para el primer bloque. El resultado es una imagen cifrada que parece completamente aleatoria y no revela ningún patrón visual de la imagen original.

### ¿Qué tan seguro es usar ECB para cifrar datos estructurados?


**Bastante** inseguro. ECB no oculta patrones en los datos, lo que permite a un atacante obtener información valiosa sobre los datos cifrados sin necesidad de descifrarlos. 